In [ ]:
%load_ext autoreload
%autoreload 2

# Import necessary modules 
import numpy as np
import pandas as pd
from itertools import product
import plotly.graph_objs as go
import plotly.express as px
from numpy.linalg import eigvalsh
from plotly.subplots import make_subplots

from rbf_volatility_surface import RBFVolatilitySurface
from smoothness_prior import RBFQuadraticSmoothnessPrior
from dataset_sabr import generate_sabr_call_options

In [ ]:
# Define the strike price list and maturity time list
strike_price_list = np.array([0.75, 0.85, 0.9, 0.95, 1.0, 1.05, 1.1, 1.2, 1.3, 1.5])
maturity_time_list = np.array([0.02, 0.08, 0.17, 0.25, 0.5, 0.75, 1.0, 1.5, 2.0, 3.0])

# Create the product grid of maturity times and strike prices
product_grid = list(product(maturity_time_list, strike_price_list))
maturity_times, strike_prices = zip(*product_grid)

# Convert to arrays for further operations
maturity_times = np.array(maturity_times)
strike_prices = np.array(strike_prices)

# Variance formula for log-uniform distribution
def log_uniform_variance(a, b):
    log_term = np.log(b / a)
    var = ((b ** 2 - a ** 2) / (2 * log_term)) - ((b - a) / log_term) ** 2
    return var

# Calculate standard deviations for maturity times and strike prices
maturity_std = np.sqrt(log_uniform_variance(maturity_time_list.min(), maturity_time_list.max()))
strike_std = np.sqrt(log_uniform_variance(strike_price_list.min(), strike_price_list.max()))

# Define the SABR model parameters
alpha = 0.20  # Stochastic volatility parameter
beta = 0.50   # Elasticity parameter
rho = -0.75   # Correlation between asset price and volatility
nu = 1.0      # Volatility of volatility parameter

# Other model parameters
risk_free_rate = np.log(1.02)  # Risk-free interest rate
underlying_price = 1.0         # Current price of the underlying asset

# Generate the dataset using the SABR model and Black-Scholes formula
call_option_dataset = generate_sabr_call_options(
    alpha=alpha,
    beta=beta,
    rho=rho,
    nu=nu,
    maturity_times=maturity_times,
    strike_prices=strike_prices,
    risk_free_rate=risk_free_rate,
    underlying_price=underlying_price
)

# Maturity times and strike prices from the previous product grid setup
hypothetical_maturity_time_list = np.logspace(np.log10(0.01), np.log10(3.1), 100)
hypothetical_strike_price_list = np.logspace(np.log10(0.7), np.log10(1.75), 100)

# Create the product grid of maturity times and strike prices
hypothetical_product_grid = list(product(hypothetical_maturity_time_list, hypothetical_strike_price_list))
hypothetical_maturity_times, hypothetical_strike_prices = zip(*hypothetical_product_grid)
hypothetical_maturity_times, hypothetical_strike_prices = np.array(hypothetical_maturity_times), np.array(hypothetical_strike_prices)

# Reshape the data for 3D surface plotting
hypothetical_maturities_grid = hypothetical_maturity_times.reshape((len(hypothetical_maturity_time_list), len(hypothetical_strike_price_list)))  
hypothetical_strikes_grid = hypothetical_strike_prices.reshape((len(hypothetical_maturity_time_list), len(hypothetical_strike_price_list)))

In [ ]:
n_roots = 350
smoothness_controller = 3.274549162877732e-05

# Initialize the RBFQuadraticSmoothnessPrior class
smoothness_prior = RBFQuadraticSmoothnessPrior(
    maturity_times=maturity_times,
    strike_prices=strike_prices,
    maturity_std=maturity_std,
    strike_std=strike_std,
    n_roots=n_roots,
    smoothness_controller=smoothness_controller,
    random_state=0,
)

In [ ]:
f_norm_regularizer = 1000000000

# Calculate the Frobenius norm of the observed implied volatility surface
data_frobenius_norm = np.linalg.norm(call_option_dataset["Implied Volatility"].values)

# Generate 100 log-spaced values for the smoothness controller (gamma)
smoothness_values = np.logspace(-6, -1, 100)

# Initialize a list to store log-likelihood values
regularized_log_likelihoods = []

# Loop through each smoothness value and calculate the log-likelihood
for gamma in smoothness_values:
    # Set the smoothness controller for the smoothness prior
    smoothness_prior.smoothness_controller = gamma
    
    # Calculate the log-likelihood for the current smoothness controller
    log_likelihood_value = smoothness_prior.log_likelihood(
        data_implied_volatilities=call_option_dataset["Implied Volatility"].values,
        data_maturity_times=call_option_dataset["Time to Maturity"].values,
        data_strike_prices=call_option_dataset["Strike Price"].values,
        risk_free_rate=risk_free_rate,
        underlying_price=underlying_price,
        noise_level=0
    )

    sqrt_expected_squared_frobenius_norm = smoothness_prior.square_root_expected_squared_frobenius_norm(
        data_implied_volatilities=call_option_dataset["Implied Volatility"].values,
        data_maturity_times=call_option_dataset["Time to Maturity"].values,
        data_strike_prices=call_option_dataset["Strike Price"].values,
        risk_free_rate=risk_free_rate,
        underlying_price=underlying_price,
    )

    regularized_log_likelihood = log_likelihood_value - \
        f_norm_regularizer * (data_frobenius_norm - sqrt_expected_squared_frobenius_norm) ** 2   
    
    # Append the log-likelihood value to the list
    regularized_log_likelihoods.append(regularized_log_likelihood)

# Convert the list of log-likelihoods to a NumPy array for easy processing
regularized_log_likelihoods = np.array(regularized_log_likelihoods)

# Find the optimal smoothness controller (the one with the maximum log-likelihood)
optimal_smoothness = smoothness_values[np.argmax(regularized_log_likelihoods)]
smoothness_prior.smoothness_controller = optimal_smoothness

# Create a Plotly figure to visualize the log-likelihoods
fig = go.Figure()

# Add a line plot for the log-likelihood values
fig.add_trace(
    go.Scatter(
        x=smoothness_values,
        y=-regularized_log_likelihoods,
        mode='lines+markers',
        name='Log-Likelihood',
        line=dict(color='royalblue', width=2),
        marker=dict(size=6)
    )
)

# Highlight the optimal smoothness controller with a vertical line
fig.add_trace(
    go.Scatter(
        x=[optimal_smoothness],
        y=[min(-regularized_log_likelihoods)],
        mode='markers',
        name=f'Optimal γ: {optimal_smoothness:.7f}',
        marker=dict(color='red', size=10, symbol='circle')
    )
)

# Update the layout for the plot
fig.update_layout(
    title="Minus Regularized Log-Likelihood vs Smoothness Controller",
    xaxis_title="Smoothness Controller (γ)",
    yaxis_title="Log-Likelihood",
    xaxis_type="log",  # Log scale for smoothness controller
    yaxis_type="log",
    height=900,
    width=900,
    legend=dict(x=0.1, y=0.9),
    title_x=0.5,
)

# Show the plot
fig.show()

# Print the optimal smoothness controller
print(f"Optimal smoothness controller: {optimal_smoothness}")
fig.write_image('figs/optimal_smoothness_controller.png', format='png', scale=4, width=900, height=900)

In [ ]:
# Calculate the prior covariance matrix
prior_covariance_matrix = smoothness_prior.prior_covariance()

# Display the covariance matrix
print("Prior Covariance Matrix:")
pd.DataFrame(prior_covariance_matrix)

In [ ]:
np.linalg.eigvalsh(prior_covariance_matrix)

In [ ]:
# Create a heatmap for the prior covariance matrix
fig1 = px.imshow(
    prior_covariance_matrix,
    labels=dict(x="Index", y="Index", color="Cov."),
    title="Smoothness Prior Covariance Matrix",
    color_continuous_scale="Viridis"
)

# Update layout for better visualization
fig1.update_layout(
    width=900,
    height=900,
    title_x=0.5,
    xaxis_title="Maturity and Strike Indices",
    yaxis_title="Maturity and Strike Indices",
)

# Show the figure
fig1.show()
fig1.write_image('figs/smoothness_prior_covariance.png', format='png', scale=4, width=900, height=900)

In [ ]:
# Compute the eigenvalues of the prior covariance matrix
eigenvalues = np.linalg.eigvalsh(prior_covariance_matrix)

# Sort eigenvalues by their absolute values in descending order
sorted_eigenvalues = np.sort(eigenvalues)[::-1]

# Calculate the cumulative explained variance
cumulative_explained_variance = np.cumsum(sorted_eigenvalues) / np.sum(sorted_eigenvalues)

# Create a bar plot for the sorted eigenvalues (original values) and a line plot for cumulative explained variance
fig2 = go.Figure()

# Bar plot for eigenvalues
fig2.add_trace(
    go.Bar(
        x=np.arange(1, len(sorted_eigenvalues) + 1),
        y=np.abs(sorted_eigenvalues),
        name="Eigenvalues",
    )
)

# Line plot for cumulative explained variance
fig2.add_trace(
    go.Scatter(
        x=np.arange(1, len(cumulative_explained_variance) + 1),
        y=cumulative_explained_variance,
        name="Cumulative Explained Variance",
        yaxis="y2",
        mode="lines",
    )
)

# Add figure layout
fig2.update_layout(
    title="Eigenvalues and Cumulative Explained Variance",
    xaxis=dict(title="Index of Eigenvalues"),
    yaxis=dict(title="Eigenvalues"),
    yaxis2=dict(
        title="Cumulative Explained Variance",
        overlaying="y",
        side="right",
        showgrid=False,
    ),
    width=900,
    height=900,
    title_x=0.5,
    legend=dict(x=0.1, y=0.9),
)

# Show the figure
fig2.show()
fig2.write_image('figs/smoothness_prior_eigenvalues_cumulative_explained_variance.png', format='png', scale=4, width=900, height=900)

In [ ]:
smoothness_prior.sample_smooth_surfaces(9)

In [ ]:
# Sample 9 surface coefficients using the smoothness prior
n_samples = 9
sampled_coefficients = smoothness_prior.sample_smooth_surfaces(n_samples)

# The constant_volatility is set to a reasonable value
constant_volatility = RBFVolatilitySurface.calculate_constant_volatility(
    call_option_dataset["Implied Volatility"],
    call_option_dataset["Time to Maturity"],
    call_option_dataset["Strike Price"],
    risk_free_rate,
    underlying_price
)

# Create the 3x3 subplots
fig = make_subplots(
    rows=3, cols=3,
    subplot_titles=[f"Sample Surface {i+1}" for i in range(n_samples)],
    specs=[[{'type': 'surface'}] * 3] * 3,
    vertical_spacing=0.075, horizontal_spacing=0.001
)

# Loop through the sampled coefficients to generate and plot each surface
for idx, coefficients in enumerate(sampled_coefficients):
    row = (idx // 3) + 1
    col = (idx % 3) + 1
    
    # Initialize the RBFVolatilitySurface class for each set of coefficients
    rbf_surface = RBFVolatilitySurface(
        coefficients=coefficients,
        maturity_times=maturity_times,
        strike_prices=strike_prices,
        maturity_std=maturity_std,
        strike_std=strike_std,
        constant_volatility=constant_volatility
    )

    # Generate the volatility surface over the product grid of times and strikes
    surface_volatilities = np.array([
        rbf_surface.implied_volatility_surface(T, K)
        for T, K in hypothetical_product_grid
    ]).reshape(len(hypothetical_maturity_time_list), len(hypothetical_strike_price_list))

    # Add the surface plot to the subplot
    fig.add_trace(
        go.Surface(
            x=hypothetical_strikes_grid,
            y=hypothetical_maturities_grid,
            z=surface_volatilities,
            colorscale='dense',
            showscale=False  # Hide the scale on individual plots
        ),
        row=row, col=col
    )

for i in range(1, 4):  # Rows
    for j in range(1, 4):  # Columns
        fig.update_scenes(
            dict(
                xaxis_title="Strike Price",
                yaxis_title="Time to Maturity",
                zaxis_title="Implied Volatility",
                camera=dict(
                    eye=dict(x=1.4, y=1.4, z=1.4)  # Controls the zoom level, adjust these to zoom out
                ),
                aspectratio=dict(x=1, y=1, z=0.8)
            ),
            row=i, col=j
        )    

# Update the layout of the figure
fig.update_layout(
    title="Sampled Volatility Surfaces from Smoothness Prior",
    height=1500, width=1500, title_x=0.5,
)

# Show the plot
fig.show()
fig.write_image('figs/sampled_surfaces_smoothness_prior.png', format='png', scale=4, width=1500, height=1500)